# Prepare Data From CSV Recording

## Load File

Data is loaded from a CSV recording file, accepted through an input prompt. This includes all positional data related to the 6 trackers (HMD, Left Controller, Right Controller, Waist, Left Foot, Right Foot).

'Data is loaded into a Pandas dataframe. The primary tracking data is then extracted, leaving extraneous data such as booleans for button presses.

The extracted columns are then concatenated into a new dataframe, and the columns are renamed for ease of reading.

The columns are reorded in the order of head/r_controller/l_controller/waist/r_foot/l_foot.

The new trimmed file is written to a directory (/test_data or /train_data), for further manipulation and loading into the model.


In [17]:
import pandas as pd


#Read in CSV
def GetRecording(path):
    recording_path = "../recordings/"
    file_name = input("Input Recording File Name")
    try:
        dataframe = pd.read_csv(recording_path + file_name + ".csv")
        return dataframe, file_name
    except: 
        print("Error Reading File: Check Spelling and Try Again")
        return 0
    
    
#Seperate each tracker to seperate dataframe

def GetColByName(dataframe):
    HMD = dataframe.loc[:, ["HMD0_tx", "HMD0_ty", "HMD0_tz"]]
    
    controller_1 = dataframe.loc[:, ['controller3_tx', 'controller3_ty', 'controller3_tz']]

    controller_2 = dataframe.loc[:, ['controller4_tx', 'controller4_ty', 'controller4_tz']]

    tracker_1 = dataframe.loc[:, ['generic7_tx', 'generic7_ty', 'generic7_tz']]

    tracker_2 = dataframe.loc[:, ['generic8_tx', 'generic8_ty', 'generic8_tz']]

    tracker_3 = dataframe.loc[:, ['generic9_tx', 'generic9_ty', 'generic9_tz']]

    joined = pd.concat([HMD,controller_1, controller_2, tracker_1 ,tracker_2 ,tracker_3], axis=1)
    return joined

def AssignTracker(dataframe):
    display(dataframe.iloc[0:1,:])
    trackerNum = 7
    for x in range(3):
        trackerStr = str(trackerNum)
        tracker = input('assign generic' + trackerStr)
        dataframe.rename(columns={'generic' + trackerStr + '_tx': tracker + '_x', 'generic' + trackerStr + '_ty': tracker + "_y", 'generic' + trackerStr + '_tz': tracker + '_z'}, inplace=True)
        trackerNum += 1
        
    controllerNum = 3
    for x in range(2):
        controllerStr = str(controllerNum)
        controller = input('assign controller' + controllerStr)
        dataframe.rename(columns={'controller' + controllerStr + '_tx': controller + '_x', 'controller' + controllerStr + '_ty': controller + "_y", 'controller' + controllerStr + '_tz': controller + '_z'}, inplace=True)
        controllerNum += 1
    dataframe.rename(columns={'HMD0_tx': 'head_x', 'HMD0_ty': 'head_y', 'HMD0_tz': 'head_z'}, inplace=True)
    return dataframe

def GetDirectory():
    choice = input("train or test data:")
    if choice == "test":
        output_path = "../test_data/"
    else:
        output_path = "../train_data/"
    return output_path

def OrderFeatures(dataframe):
    head = dataframe.loc[:, ['head_x', 'head_y', 'head_z']]
    l_controller = dataframe.loc[:, ['l_controller_x', 'l_controller_y', 'l_controller_z']]
    r_controller = dataframe.loc[:, ['r_controller_x', 'r_controller_y', 'r_controller_z']]
    waist = dataframe.loc[:, ['waist_x', 'waist_y', 'waist_z']]
    r_foot = dataframe.loc[:, ['r_foot_x', 'r_foot_y', 'r_foot_z']]
    l_foot = dataframe.loc[:, ['l_foot_x', 'l_foot_y', 'l_foot_z']]
    reordered = pd.concat([head , r_controller, l_controller, waist, r_foot, l_foot], axis=1)
    return reordered

    
    
def WriteOutput(path, dataframe, filename):
    output_file = path + filename + "_trimmed.csv"
    dataframe.to_csv(output_file, index = False)
    print(file_name + " output to " + path)
    
    

# Run data trimming functions

In [41]:
recording_path = " ../recordings"

dataframe, file_name = GetRecording(recording_path)
joined = GetColByName(dataframe)
renamed = AssignTracker(joined)
path = GetDirectory()
reordered = OrderFeatures(renamed)
WriteOutput(path, reordered, file_name)


Input Recording File Name leg_raise_5


,HMD0_tx,HMD0_ty,HMD0_tz,controller3_tx,controller3_ty,controller3_tz,controller4_tx,controller4_ty,controller4_tz,generic7_tx,generic7_ty,generic7_tz,generic8_tx,generic8_ty,generic8_tz,generic9_tx,generic9_ty,generic9_tz
0,1.964998,158.131058,3.838474,-18.907917,92.860924,-9.813828,29.159588,80.373276,2.166218,3.899813,97.88607,6.474656,21.754217,11.036432,13.468838,-12.954676,11.488569,19.045788


assign generic7 waist
assign generic8 r_foot
assign generic9 l_foot
assign controller3 l_controller
assign controller4 r_controller
train or test data: test


leg_raise_5 output to ../test_data/


# Data Normalization

## Data Scaling

The new CSV is loaded into memory, chosen through an input prompt
The data is then split between the features (the HMD and controller tracking data), and the labels (the waist and foot trackers).
These are loaded into Numpy arrays to peform normaliztion. The output from OpenVR Recorder is upscaled by 100. To correct this the array is divided by 100

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
#import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


output_path = "../trim_output/"


#read in formatted CSV
def ReadCSV(path):
    file_name = input("Input File Name")
    try:
        dataframe = pd.read_csv(path + file_name + ".csv")
        print("Dataframe created")
    except:
        print("Error Reading File")
    return dataframe

def SplitFeaturesLabels(dataframe):
    x = dataframe.iloc[:, 0:9]
    y = dataframe.iloc[:, 9:18]
    return x, y

#Load data into Numpy array
def LoadArray(x, y):
    x_array = np.array(x)
    y_array = np.array(y)
    return x_array, y_array


def NormalizeValues (x, y):
    x =  np.divide(x, 100)
    y =  np.divide(y, 100)
    return x, y

def SampleSize(x, y):
    x_samples = x[0:600,:]
    y_samples = y[0:600,:]
    return x_samples, y_samples

def RoundValues(x, y): 
    x_rounded = np.around(x, 3)
    y_rounded = np.around(y, 3)
    return x_rounded, y_rounded



    
    

    





# Create Training Data

In [45]:
train_path = "../train_data/"
#load train data from csv
train_dataframe = ReadCSV(train_path)

#split features and labels into seperate dataframes
x_train_df, y_train_df = SplitFeaturesLabels(train_dataframe)

#convert features and labels to numpy array
x_train, y_train = LoadArray(x_train_df, y_train_df)

#Divide values in array by 100
x_samples, y_samples = NormalizeValues(x_train, y_train)

print(x_samples.shape, x_samples)
print(y_samples.shape, y_samples)

#x_train, x_test, y_train, y_test = train_test_split(x_train_normalized, y_train_normalized)





Input File Name walking_7_trimmed


Dataframe created
(600, 9) [[0.56456936 1.65256363 0.82371849 ... 0.35004543 0.81725983 0.93927994]
 [0.56244183 1.6511261  0.82760582 ... 0.35082798 0.81521797 0.94517693]
 [0.56021255 1.64923187 0.8309227  ... 0.3510099  0.81342445 0.95069672]
 ...
 [0.42225151 1.55701126 0.6327589  ... 0.21533564 0.75101753 0.71076973]
 [0.41390022 1.55681183 0.62624306 ... 0.20336998 0.75672852 0.69413803]
 [0.40570499 1.55652176 0.6195657  ... 0.19236187 0.76251534 0.67823952]]
(600, 9) [[0.56388847 0.9584938  0.82307373 ... 0.6701664  0.12568272 1.02205978]
 [0.5649939  0.95908966 0.8268927  ... 0.67017311 0.12555384 1.02190422]
 [0.5658884  0.95991432 0.83057884 ... 0.67164154 0.12405634 1.02306038]
 ...
 [0.4459425  0.94995354 0.64034004 ... 0.39146503 0.11636929 0.66756081]
 [0.43529736 0.9514679  0.63556534 ... 0.39199577 0.115468   0.66869583]
 [0.42703381 0.95304367 0.63249866 ... 0.39176373 0.11532265 0.66892319]]


# Scale Values

In [ ]:
from sklearn.preprocessing import MinMaxScaler

print(x_samples.max())
print(x_samples.min())

print(y_samples.max())
print(y_samples.min())

scaler =MinMaxScaler()
print(x_samples[0:1])
scaled = scaler.fit(x_samples)
print(scaler.transform(x_samples[0:1]))
print(scaler.inverse_transform(x_samples[0:1]))


# Reshape Training Data

In [5]:

def ReshapeData(x, y):
    x_reshaped = np.expand_dims(x, axis=1)
    y_reshaped = np.expand_dims(y, axis=1)

    return x_reshaped, y_reshaped


In [46]:
x_train, y_train = ReshapeData(x_samples, y_samples)

print(x_train.shape, y_train.shape)

print(x_train.shape[1])

print(x_train.shape[2])


(600, 1, 9) (600, 1, 9)
1
9


# Create Test / Validation Data

In [42]:
#Create a single test data file

test_path = "../test_data/"


test_dataframe = ReadCSV(test_path)

#split features and labels into seperate dataframes
x_test_df, y_test_df = SplitFeaturesLabels(test_dataframe)

#convert features and labels to numpy array
x_test, y_test = LoadArray(x_test_df, y_test_df)

#Divide values in array by 100
x_test_normalized, y_test_normalized = NormalizeValues(x_test, y_test)

x_test_samples, y_test_samples = SampleSize(x_test_normalized, y_test_normalized)

x_test, y_test = ReshapeData(x_test_samples, y_test_samples)

print(x_test.shape, y_test.shape)

print(x_test.shape[1])

KeyboardInterrupt: Interrupted by user

# Combine Test and Train datasets

Combine all data sets in /train_data and /test_data into one, for more samples when training 

In [7]:
#import modules 

import os
import pandas as pd
import numpy as np


In [8]:

#Combine all datasets in a directory into one dataframe
def CombineDatasets(path):
    data_list = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"):
            df = pd.read_csv(path + filename)
            
            data_list.append(df)
         

    data_df = pd.concat(data_list, axis=0, ignore_index=True)
    return data_df



In [9]:

#combine data in a directory into two lists of x and y features
def DatasetsLists(path):
    x_list = []
    y_list = []
    for file in os.listdir(path):
        filename = os.fsdecode(file)
        print(filename)
        if filename.endswith(".csv"):
            df = pd.read_csv(path + filename)
            x_features, y_features = SplitFeaturesLabels(df)
            x_train, y_train = LoadArray(x_features, y_features)
            x_normalized, y_normalized = NormalizeValues(x_train, y_train)
            x_reshape, y_reshape = ReshapeData(x_normalized, y_normalized)
            x_list.append(x_reshape)
            y_list.append(y_reshape)
    return x_list, y_list



In [56]:
train_path = "../train_data/"
x, y = DatasetsLists(train_path)


.ipynb_checkpoints
arm_raise_1_trimmed.csv
arm_raise_3_trimmed.csv
boxing_1_trimmed.csv
crouch_walking_1_trimmed.csv
jumping_1_trimmed.csv
jumping_2_trimmed.csv
jumping_3_trimmed.csv
leg_raise_1_trimmed.csv
leg_raise_2_trimmed.csv
leg_raise_3_trimmed.csv
leg_raise_4_trimmed.csv
picking_up_1_trimmed.csv
running_on_spot_2_trimmed.csv
sitting_on_floor_1_trimmed.csv
sitting_standing_1_trimmed.csv
sitting_standing_2_trimmed.csv
sitting_standing_3_trimmed.csv
walking_1_train.csv
walking_2_test.csv
walking_3_trimmed.csv
walking_5_trimmed.csv
walking_6_trimmed.csv
walking_7_trimmed.csv


Combine all data in the training data directory 

In [43]:
train_path = "../train_data/"

combined_train_dataframe = CombineDatasets(train_path)

x_train, y_train = SplitFeaturesLabels(combined_train_dataframe)
        
x_train_arr, y_train_arr = LoadArray(x_train, y_train)

#Divide values in array by 100
x_train_normalized, y_train_normalized = NormalizeValues(x_train_arr, y_train_arr)

x_train, y_train = ReshapeData(x_train_normalized, y_train_normalized)

print(x_train.shape, y_train.shape)


(16108, 1, 9) (16108, 1, 9)


Combine all data in the test data directory

In [44]:
test_path = "../test_data/"

combined_test_dataframe = CombineDatasets(test_path)

x_test, y_test = SplitFeaturesLabels(combined_test_dataframe)
        
x_test_arr, y_test_arr = LoadArray(x_test, y_test)

x_test_normalized, y_test_normalized = NormalizeValues(x_test_arr, y_test_arr)

x_test, y_test = ReshapeData(x_test_normalized, y_test_normalized)

print(x_test.shape)
print(x_test[0:1,:,:])
print(x_test_normalized.shape)


(5400, 1, 9)
[[[ 0.07488835  1.60079437 -0.06116605  0.29804235  0.82701195
   -0.13233584 -0.16509771  0.80914047 -0.10186529]]]
(5400, 9)


# Model Creation and Training

In [79]:
#import required modules

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization
import tensorflow as tf
from keras.layers import LSTM, GRU
from tensorflow.keras.optimizers import SGD
from keras.layers.core import Dense, Activation, Dropout
from keras.losses import SparseCategoricalCrossentropy

tf.keras.backend.set_floatx('float64')


# Model Functions

In [164]:
#Fit model to two equally sized lists of x features and y labels 

def FitToList(x, y, model, epoch, b_size, verbose):
    for i in range(len(x)):
        print(i)
        print(x[i].shape)
        print(y[i].shape)
        model.fit(x[i], y[i], epochs=epoch,batch_size=b_size, verbose=verbose)
        
 # fit the model to a given set of features (x) and labels (y)    
def FitModel(x, y, model, epoch, b_size):
    model.fit(x, y, validation_split=0.33, epochs=epoch,batch_size = b_size)

    
def EvaluateModel(x, y, model):
    metrics = model.evaluate(x, y, batch_size=256)
    
    
    
def PredictModel(x, y, model):
    predictions = model.predict(x)
    y_reshaped = y.reshape(-1,9)
    return predictions, y_reshaped


def DisplayPredictions(prediction, actual, range_1, range_2):
    if range_2 <= 0:
        r_1 = 0
        r_2 = len(prediction)
    else:
        r_1 = range_1
        r_2 = range_2
    print("predictions shape:", prediction.shape)
    prediction_df = pd.DataFrame(prediction, columns=["Waist_X", "Waist_Y", "Waist_Z", "R_Foot_X", "R_Foot_Y", "R_Foot_Z", "L_Foot_X", "L_Foot_Y", "L_Foot_Z"])
    actual_df = pd.DataFrame(actual, columns=["Waist_X", "Waist_Y", "Waist_Z", "R_Foot_X", "R_Foot_Y", "R_Foot_Z", "L_Foot_X", "L_Foot_Y", "L_Foot_Z"])
    print("Actual Values")
    display(actual_df[r_1:r_2])
    print("Predicited Values")
    display(prediction_df[r_1:r_2])
    return actual_df, prediction_df
    

        

# Model 1

In [8]:
sgd = SGD(learning_rate=0.001, momentum=0.8, decay=0.999, nesterov=False)

tf.keras.backend.set_floatx('float64')

model = Sequential()
model.add(GRU(16, return_sequences=True, input_shape=(5, 9)))
model.add(Dropout(0.2))
model.add(GRU(16, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(9, activation = "linear"))

model.compile(loss='mse', optimizer="adam")

print ('model compiled')

print (model.summary())

model compiled
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 1, 16)             1296      
                                                                 
 dropout (Dropout)           (None, 1, 16)             0         
                                                                 
 gru_1 (GRU)                 (None, 16)                1632      
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 9)                 153       
                                                                 
Total params: 3,081
Trainable params: 3,081
Non-trainable params: 0
_________________________________________________________________
None


# Model 2 

In [115]:
model_2 = Sequential()

#add one GRU layer of 64 cellss with input shape 1,9
model_2.add(GRU(64, input_shape=(1, 9)))
#model_2.add(Dropout(0.5))

#Add Batch Normalization layer
#model_2.add(BatchNormalization())


#Add dropout layer
#model_2.add(Dropout(0.2))

#Add Dense layer with 9 outputs
model_2.add(Dense(9))

print(model_2.summary())

#Compile model
model_2.compile(
    loss="mse",
    optimizer="adam",
    metrics=["accuracy"],
)



Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_12 (GRU)                (None, 64)                14400     
                                                                 
 dense_7 (Dense)             (None, 9)                 585       
                                                                 
Total params: 14,985
Trainable params: 14,985
Non-trainable params: 0
_________________________________________________________________
None


In [112]:
#Fit single dataset to model
FitModel(x_train, y_train, model_2, 200, 1)

Epoch 1/200
401/401 [==============================] - 2s 2ms/step - loss: 0.0430 - accuracy: 0.8404 - val_loss: 0.0129 - val_accuracy: 0.8191
Epoch 2/200
401/401 [==============================] - 1s 1ms/step - loss: 0.0073 - accuracy: 0.9127 - val_loss: 0.0125 - val_accuracy: 0.5628
Epoch 3/200
401/401 [==============================] - 1s 1ms/step - loss: 0.0068 - accuracy: 0.9177 - val_loss: 0.0104 - val_accuracy: 0.6231
Epoch 4/200
401/401 [==============================] - 1s 1ms/step - loss: 0.0064 - accuracy: 0.9377 - val_loss: 0.0099 - val_accuracy: 0.6382
Epoch 5/200
401/401 [==============================] - 0s 1ms/step - loss: 0.0059 - accuracy: 0.9501 - val_loss: 0.0103 - val_accuracy: 0.5729
Epoch 6/200
401/401 [==============================] - 0s 1ms/step - loss: 0.0055 - accuracy: 0.9426 - val_loss: 0.0095 - val_accuracy: 0.6583
Epoch 7/200
401/401 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 0.9476 - val_loss: 0.0095 - val_accuracy: 0.7035

In [116]:
#Fit list of datasets to model

FitToList(x, y, model_2, 20, 1, 1)

0
(905, 1, 9)
(905, 1, 9)
Epoch 1/20
905/905 [==============================] - 3s 2ms/step - loss: 0.0021 - accuracy: 0.9945
Epoch 2/20
905/905 [==============================] - 1s 1ms/step - loss: 2.1529e-05 - accuracy: 1.0000
Epoch 3/20
905/905 [==============================] - 1s 997us/step - loss: 1.9848e-05 - accuracy: 1.0000
Epoch 4/20
905/905 [==============================] - 1s 940us/step - loss: 2.0158e-05 - accuracy: 1.0000
Epoch 5/20
905/905 [==============================] - 1s 853us/step - loss: 2.0968e-05 - accuracy: 1.0000
Epoch 6/20
905/905 [==============================] - 1s 843us/step - loss: 2.2221e-05 - accuracy: 1.0000
Epoch 7/20
905/905 [==============================] - 1s 854us/step - loss: 2.1092e-05 - accuracy: 1.0000
Epoch 8/20
905/905 [==============================] - 1s 848us/step - loss: 2.0868e-05 - accuracy: 1.0000
Epoch 9/20
905/905 [==============================] - 1s 853us/step - loss: 1.9991e-05 - accuracy: 1.0000
Epoch 10/20
905/905 [=======

In [117]:
EvaluateModel(x_test, y_test, model_2)

22/22 [==============================] - 0s 3ms/step - loss: 0.1530 - accuracy: 0.7135
Loss:  0.1529950911137793


In [178]:
pred, actual = PredictModel(x_test, y_test, model_2)
print(actual.shape)
print(pred.shape)
toWriteActual, toWritePred = DisplayPredictions(pred, actual, 4000, 4050)



(5400, 9)
(5400, 9)
predictions shape: (5400, 9)
Actual Values


,Waist_X,Waist_Y,Waist_Z,R_Foot_X,R_Foot_Y,R_Foot_Z,L_Foot_X,L_Foot_Y,L_Foot_Z
4000,0.568578,0.948957,0.022448,0.443972,0.099626,0.040721,0.847469,0.105724,-0.260403
4001,0.559899,0.950990,0.032180,0.444332,0.099212,0.041662,0.843127,0.109440,-0.254307
4002,0.552315,0.952658,0.041153,0.445992,0.098248,0.041867,0.839128,0.115232,-0.247011
4003,0.545255,0.954177,0.050479,0.446416,0.097923,0.042506,0.833094,0.123043,-0.237844
4004,0.538411,0.956142,0.058618,0.445989,0.098010,0.042564,0.824971,0.134583,-0.227277
4005,0.532336,0.957229,0.066987,0.446254,0.098038,0.043005,0.815174,0.146172,-0.213108
4006,0.526953,0.958167,0.075301,0.446580,0.097753,0.043268,0.802805,0.157194,-0.196686
4007,0.521210,0.958752,0.083070,0.446938,0.097715,0.043523,0.788901,0.167540,-0.179047
4008,0.516434,0.959480,0.089992,0.447252,0.097746,0.043566,0.773595,0.175675,-0.159534
4009,0.511045,0.960276,0.097480,0.447538,0.097748,0.043522,0.756670,0.183696,-0.137797


Predicited Values


,Waist_X,Waist_Y,Waist_Z,R_Foot_X,R_Foot_Y,R_Foot_Z,L_Foot_X,L_Foot_Y,L_Foot_Z
4000,0.976676,1.224183,0.289238,-0.496335,0.053867,-0.043498,0.865819,0.463140,-0.526091
4001,0.950934,1.210933,0.300699,-0.506135,0.052749,-0.019599,0.873998,0.448951,-0.522736
4002,0.925361,1.197943,0.312225,-0.514672,0.052463,0.002984,0.880861,0.434433,-0.517868
4003,0.898110,1.184754,0.323552,-0.518997,0.052893,0.025483,0.885049,0.419374,-0.509630
4004,0.870596,1.172180,0.333662,-0.519373,0.054445,0.046371,0.886216,0.404494,-0.499613
4005,0.842546,1.159517,0.344233,-0.518455,0.056463,0.066589,0.885809,0.388882,-0.487463
4006,0.815824,1.147585,0.354245,-0.516732,0.059069,0.085118,0.883878,0.373643,-0.475693
4007,0.788414,1.135978,0.363500,-0.511211,0.062655,0.103100,0.878378,0.358224,-0.462178
4008,0.762003,1.125182,0.371663,-0.503980,0.066878,0.119850,0.870706,0.343416,-0.449404
4009,0.735495,1.114798,0.379169,-0.493962,0.071835,0.136177,0.860058,0.328665,-0.436027


# Write Results to File 

In [179]:
results_path = "../results/"

for column in toWriteActual.columns:
    toWriteActual.rename(columns={column : "Actual_" + column}, inplace=True)

for column in toWritePred.columns:
    toWritePred.rename(columns={column : "Pred_" + column}, inplace=True)

results = pd.concat([toWriteActual, toWritePred], axis=1)
display(results)

output = results_path+ "output.csv"
results.to_csv(output, index = False)



,Actual_Waist_X,Actual_Waist_Y,Actual_Waist_Z,Actual_R_Foot_X,Actual_R_Foot_Y,Actual_R_Foot_Z,Actual_L_Foot_X,Actual_L_Foot_Y,Actual_L_Foot_Z,Pred_Waist_X,Pred_Waist_Y,Pred_Waist_Z,Pred_R_Foot_X,Pred_R_Foot_Y,Pred_R_Foot_Z,Pred_L_Foot_X,Pred_L_Foot_Y,Pred_L_Foot_Z
0,0.060439,1.022620,-0.098646,0.296409,0.111177,-0.002579,-0.105350,0.107512,0.008089,0.220943,1.013334,0.143499,-0.211455,0.113909,0.241544,0.342301,0.102949,-0.439054
1,0.060260,1.022620,-0.098646,0.296409,0.111177,-0.002579,-0.105350,0.107512,0.008089,0.220573,1.013352,0.143619,-0.211553,0.114130,0.241542,0.342526,0.102511,-0.438521
2,0.059955,1.022620,-0.098769,0.296418,0.111177,-0.002579,-0.105350,0.107433,0.008048,0.220180,1.013364,0.143694,-0.211643,0.114310,0.241550,0.342751,0.102095,-0.438064
3,0.059620,1.022658,-0.099001,0.296458,0.111177,-0.002579,-0.105350,0.107417,0.007960,0.219799,1.013369,0.143689,-0.211898,0.114430,0.241617,0.343008,0.101757,-0.437876
4,0.059283,1.022695,-0.099210,0.296351,0.111177,-0.002579,-0.105444,0.107411,0.007911,0.219224,1.013377,0.143643,-0.212029,0.114606,0.241705,0.343034,0.101401,-0.437544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5395,-0.266650,1.028392,-0.104049,-0.055547,0.211484,0.045802,-0.325084,0.122200,0.065459,-0.332323,0.983205,-0.058999,-0.325814,0.159368,-0.105212,-0.483079,0.121700,-0.202876
5396,-0.263761,1.024829,-0.101151,-0.055403,0.215707,0.044948,-0.325417,0.121903,0.065412,-0.333360,0.983648,-0.057928,-0.343283,0.162712,-0.114889,-0.477255,0.120932,-0.204384
5397,-0.271885,1.032776,-0.107868,-0.054945,0.219580,0.044346,-0.325417,0.121905,0.065404,-0.334303,0.984181,-0.056783,-0.360225,0.166057,-0.124497,-0.470918,0.120186,-0.205555
5398,-0.268666,1.028839,-0.104847,-0.054282,0.222948,0.044158,-0.325470,0.121905,0.065396,-0.334956,0.984779,-0.055500,-0.376179,0.169332,-0.133578,-0.464236,0.119449,-0.206404
